In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Get HTML from website
page = get('https://www.lacourt.org/judicialofficers/ui/SearchResult.aspx')

page.status_code # if starts with 2, GET was successful, if 4 or 5, it's an error
page.content # provides html content of page
page.headers # look at headers

{'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Server': 'Microsoft-IIS/7.0', 'Set-Cookie': 'ASP.NET_SessionId=s3idgm4n2ioie4i4rzzurgje; path=/; HttpOnly', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Methods': 'GET,POST,PUT', 'Access-Control-Allow-Origin': 'https://crmportal.lacourt.org', 'Access-Control-Max-Age': '86400', 'Date': 'Tue, 08 Jan 2019 18:01:20 GMT', 'Content-Length': '384688'}

In [7]:
# Creaate beautifulsoup object from html
soup = BeautifulSoup(page.content)

In [21]:
# Getting column names from table
headrow = table.find_all('thead') # gets first row in body of table
header_list = []
for item in headtuples:
    header_list.append(item.string)

header_list

['Name', 'Title', 'Location', 'Dept', 'Phone']

In [18]:
# Create pandas dataframe
df = pd.DataFrame(columns = header_list)

# Add rows to dataframe
for row in table:
    tuples = row.find_all('td')
    tuple_list = []
    for item in tuples:
        tuple_list.append(item.string)
    if len(tuple_list) > 0:
        df_row = pd.Series(tuple_list, index = header_list)
        df = df.append(df_row, ignore_index = True)

df.head()

,Name,Title,Location,Dept,Phone
0,"Abzug, Michael D.",Judge,Clara Shortridge Foltz Criminal Justice Center,112,(213) 628-7412
1,"Acevedo, Victor M.",Commissioner,Compton,007,(310) 761-7987
2,"Aceves, Efrain M.",Judge,Clara Shortridge Foltz Criminal Justice Center,046,(213) 628-7746
3,"Adams, Rashida A.",Judge,Edmund D. Edelman Children's Court,423,(323) 307-8023
4,"Aenlle-Rocha, Fernando L.",Judge,East Los Angeles,006,(323) 780-2011


### Get links in table

In [22]:
# Find links in each row of table
court_links = []
    
for row in table:
    links = row.find_all('a') # returns a Resultset list (there should only be one bc only one link in row for courthouse)
    if len(links) > 0:
        court_links.append(links[0]['href'])

In [26]:
# Append links as new column to df
df['Link'] = court_links

df.head()

,Name,Title,Location,Dept,Phone,Link
0,"Abzug, Michael D.",Judge,Clara Shortridge Foltz Criminal Justice Center,112,(213) 628-7412,../../Courthouse/info/CCB
1,"Acevedo, Victor M.",Commissioner,Compton,007,(310) 761-7987,../../Courthouse/info/COM
2,"Aceves, Efrain M.",Judge,Clara Shortridge Foltz Criminal Justice Center,046,(213) 628-7746,../../Courthouse/info/CCB
3,"Adams, Rashida A.",Judge,Edmund D. Edelman Children's Court,423,(323) 307-8023,../../Courthouse/info/CCJ
4,"Aenlle-Rocha, Fernando L.",Judge,East Los Angeles,006,(323) 780-2011,../../Courthouse/info/ELA


### Write to CSV

In [29]:
df.to_csv('../data/judicialofficers.csv', index=False)